In [68]:
import pandas as pd
import os

pasta = '/home/ezequiel/Desktop/job_gabriel'
df_a = pd.read_csv(os.path.join(pasta,'resultado.csv',  ),index_col='Unnamed: 0')
df_b = pd.read_csv(os.path.join(pasta,'resultado_principais.csv',  ),index_col='Unnamed: 0')


df_c = df_a[df_a['incidente_ou_execucao'].notnull()]
dic_conversao = {}
for _, row in df_c.iterrows():
    dic_conversao[row['processo_principal']] = row['cnj']
    
df_b['NUMERO_DA_EXECUCAO'] = df_b['cnj'].apply(lambda x : dic_conversao[x])

df_a = df_a[df_a['incidente_ou_execucao'].isnull()]

df = df_a.append(df_b, sort=True)

df.drop_duplicates(subset=['cnj'], inplace=True)

del df['movimentacoes']
del df['partes']
df.dropna(how='all', axis=1, inplace=True)
w = pd.ExcelWriter(os.path.join(pasta, 'levantamento2.xlsx'))
df.to_excel(w)
w.save()

In [70]:
w = pd.ExcelWriter(os.path.join(pasta, 'levantamento2.xlsx'))
df.to_excel(w)
w.save()

In [12]:
# import pandas as pd
# import os
# xlsx = '/home/ezequiel/Downloads/Lista DEPRE apurada 01-07-2019.xlsx'

# df = pd.read_excel(xlsx,header=2)
# df['consultado'] = False

# pasta = '/home/ezequiel/Desktop/job_gabriel'

# df.to_csv(os.path.join(pasta, 'base.csv'))

In [42]:
from bs4 import BeautifulSoup as bs
import requests
import re
from unidecode import unidecode
import datetime
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

s = requests.session()

df_base = pd.read_csv(os.path.join(pasta, 'base.csv'), index_col='Unnamed: 0')


if os.path.isfile(os.path.join(pasta, 'resultado.csv')):
    df = pd.read_csv(os.path.join(pasta, 'resultado.csv'), index_col='Unnamed: 0')
    df_base = df_base[df_base.index > df.index.max()]
else:
    df = pd.DataFrame()
    


for _, row in df_base.iterrows():
    
    entrada = {'cnj':row['Nº Autos']}

    params = {
    'conversationId': '',
    'dadosConsulta.localPesquisa.cdLocal': '-1',
    'cbPesquisa': 'NUMPROC',
    'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
    'numeroDigitoAnoUnificado': '',
    'foroNumeroUnificado:': '',
    'dadosConsulta.valorConsultaNuUnificado': row['Nº Autos'],
    'dadosConsulta.valorConsulta': '',
    'uuidCaptcha': '',
    }
    u = 'https://esaj.tjsp.jus.br/cpopg/search.do'

    r = s.get(u, params=params, verify=False)

    sp = bs(r.content, 'html5lib')

    if "mostrar: 'true'" in str(sp):
        entrada['segredo_de_justica'] = True

    else:
        # diversos

        for item in sp.findAll(attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
            if item.get('for') == 'tag.dados.rotulo.principal':
                entrada['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

            elif item.get_text() == 'Distribuição:':
                entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                rp = item.findNext('span').findNext()
                if rp.find('span') is not None:
                    entrada['vara'] = rp.find('span').get_text().strip()

            elif item.get_text() == 'Apensado ao:':
                entrada[item.get_text().strip().replace(':', '')] = item.findNext(
                    attrs={'class': 'processoPaiApenso'}).get_text().strip()

            elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
                entrada['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
            else:
                entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            if item.get_text() == 'Recebido em:':
                entrada['vara'] = item.findNext('span').findNext('span').get_text()

        for i in sp.findAll('span', attrs={'class': 'labelClass'}):
            entrada[i.get_text().replace(':', '')] = i.next.next.strip()

        # extinto_suspenso
        if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
            entrada['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

        if 'Distribuição' in entrada.keys():

            entrada['data_inicial'] = entrada['Distribuição'][:10]
        elif 'Recebido em' in entrada.keys():
            entrada['data_inicial'] = entrada['Recebido em'][:10]

        if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
            entrada['tramitacao_prioritaria'] = 'Tramitação prioritária'
        if 'Valor da ação' in entrada.keys():
            entrada['Valor da ação'] = re.sub(r'\s\s+', ' ', entrada['Valor da ação'])
            if re.match(r'(.*?)\s', entrada['Valor da ação']) is not None:
                entrada['moeda'] = re.match(r'(.*?)\s', entrada['Valor da ação']).group(1)
            if re.search(r'\s(.*)', entrada['Valor da ação']) is not None:
                rp = re.search(r'\s(.*)', entrada['Valor da ação']).group(1).strip()
                entrada['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

            del entrada['Valor da ação']

        # partes
        coleta = []
        controle = []

        for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top','style': 'padding-bottom: 5px'}):
            rp = {}
            if item.findNext('td').next.strip() not in controle:
                controle += [item.findNext('td').next.strip()]
                rp['nome'] = item.findNext('td').next.strip()
                rp['posicao'] = item.get_text().strip().replace(':', '')

                procuradores = []

                for subitem in item.findNextSibling().findAll('span'):
                    classe = subitem.get_text().replace(':', '').strip()
                    nome = subitem.next.next.strip()
                    procuradores += [[classe, nome]]

                classes = list(set([i[0] for i in procuradores]))
                subcoleta = {}

                for classe in classes:
                    nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                    subcoleta[classe] = nome_procurador

                if len(subcoleta) > 0:
                    rp['procuradores'] = subcoleta
                coleta += [rp]

        if len(coleta) > 0:
            entrada['partes'] = coleta

        # movimentacoes
        coleta = []
        lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
        r = list(range(len(lista) - 1, -1, -1))

        for n, item in enumerate(sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})[::-1]):

            mov = {'ordem': n,
                   'data': item.get_text().strip()}

            if len(item.findNext('td').findChildren()) > 0:
                mov['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
                mov['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
            else:

                mov['tag'] = item.findNext('td').findNext('td').next.strip()
                mov['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
            coleta += [mov]

        if len(coleta) > 0:
            entrada['movimentacoes'] = coleta

    entrada = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in entrada.items() if v}

    # formatacoes e salvar

    if 'data_inicial' in entrada.keys():

        entrada['data_inicial'] = datetime.datetime.strptime(entrada['data_inicial'], '%d/%m/%Y').date()


    dfp = pd.DataFrame([entrada])
    dfp = dfp.applymap(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x) 
    dfp.index = [_]
    df = df.append(dfp, sort=True)
    
    if _ % 250 == 0:
        df.to_csv(os.path.join(pasta, 'resultado.csv'))
    
    print(_)
df.to_csv(os.path.join(pasta, 'resultado.csv'))


16251
16252
16253
16254
16255
16256
16257
16258
16259
16260
16261
16262
16263
16264
16265
16266
16267
16268
16269
16270
16271
16272
16273
16274
16275
16276
16277
16278
16279
16280
16281
16282
16283
16284
16285
16286
16287
16288
16289
16290
16291
16292
16293
16294
16295
16296
16297
16298
16299
16300
16301
16302
16303
16304
16305
16306
16307
16308
16309
16310
16311
16312
16313
16314
16315
16316
16317
16318
16319
16320
16321
16322
16323
16324
16325
16326
16327
16328
16329
16330
16331
16332
16333
16334
16335
16336
16337
16338
16339
16340
16341
16342
16343
16344
16345
16346
16347
16348
16349
16350
16351
16352
16353
16354
16355
16356
16357
16358
16359
16360
16361
16362
16363
16364
16365
16366
16367
16368
16369
16370
16371
16372
16373
16374
16375
16376
16377
16378
16379
16380
16381
16382
16383
16384
16385
16386
16387
16388
16389
16390
16391
16392
16393
16394
16395
16396
16397
16398
16399
16400
16401
16402
16403
16404
16405
16406
16407
16408
16409
16410
16411
16412
16413
16414
16415
16416
1641

17617
17618
17619
17620
17621
17622
17623
17624
17625
17626
17627
17628
17629
17630
17631
17632
17633
17634
17635
17636
17637
17638
17639
17640
17641
17642
17643
17644
17645
17646
17647
17648
17649
17650
17651
17652
17653
17654
17655
17656
17657
17658
17659
17660
17661
17662
17663
17664
17665
17666
17667
17668
17669
17670
17671
17672
17673
17674
17675
17676
17677
17678
17679
17680
17681
17682
17683
17684
17685
17686
17687
17688
17689
17690
17691
17692
17693
17694
17695
17696
17697
17698
17699
17700
17701
17702
17703
17704
17705
17706
17707
17708
17709
17710
17711
17712
17713
17714
17715
17716
17717
17718
17719
17720
17721
17722
17723
17724
17725
17726
17727
17728
17729
17730
17731
17732
17733
17734
17735
17736
17737
17738
17739
17740
17741
17742
17743
17744
17745
17746
17747
17748
17749
17750
17751
17752
17753
17754
17755
17756
17757
17758
17759
17760
17761
17762
17763
17764
17765
17766
17767
17768
17769
17770
17771
17772
17773
17774
17775
17776
17777
17778
17779
17780
17781
17782
1778

In [58]:
df_base['processo_principal']

2388     0408586-12.1994.8.26.0053
2428     0808212-38.1988.8.26.0053
3136     0010827-43.2002.8.26.0053
4356     0428950-29.1999.8.26.0053
5275     0428705-18.1999.8.26.0053
                   ...            
17990    0025051-49.2003.8.26.0053
17991    0121832-94.2007.8.26.0053
17992    0121832-94.2007.8.26.0053
17994    0010633-09.2003.8.26.0053
18007    0424823-48.1999.8.26.0053
Name: processo_principal, Length: 4966, dtype: object

In [62]:
from bs4 import BeautifulSoup as bs
import requests
import re
from unidecode import unidecode
import datetime
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

s = requests.session()

df_base = pd.read_csv(os.path.join(pasta, 'resultado.csv'), index_col='Unnamed: 0')
df_base = df_base[df_base['incidente_ou_execucao'].notnull()]

if os.path.isfile(os.path.join(pasta, 'resultado_principais.csv')):
    df = pd.read_csv(os.path.join(pasta, 'resultado_principais.csv'), index_col='Unnamed: 0')
    df_base = df_base[df_base.index > df.index.max()]
else:
    df = pd.DataFrame()
    


for _, row in df_base.iterrows():
    
    entrada = {'cnj':row['processo_principal']}

    params = {
    'conversationId': '',
    'dadosConsulta.localPesquisa.cdLocal': '-1',
    'cbPesquisa': 'NUMPROC',
    'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
    'numeroDigitoAnoUnificado': '',
    'foroNumeroUnificado:': '',
    'dadosConsulta.valorConsultaNuUnificado': row['processo_principal'],
    'dadosConsulta.valorConsulta': '',
    'uuidCaptcha': '',
    }
    u = 'https://esaj.tjsp.jus.br/cpopg/search.do'

    r = s.get(u, params=params, verify=False)

    sp = bs(r.content, 'html5lib')

    if "mostrar: 'true'" in str(sp):
        entrada['segredo_de_justica'] = True

    else:
        # diversos

        for item in sp.findAll(attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
            if item.get('for') == 'tag.dados.rotulo.principal':
                entrada['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

            elif item.get_text() == 'Distribuição:':
                entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                rp = item.findNext('span').findNext()
                if rp.find('span') is not None:
                    entrada['vara'] = rp.find('span').get_text().strip()

            elif item.get_text() == 'Apensado ao:':
                entrada[item.get_text().strip().replace(':', '')] = item.findNext(
                    attrs={'class': 'processoPaiApenso'}).get_text().strip()

            elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
                entrada['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
            else:
                entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            if item.get_text() == 'Recebido em:':
                entrada['vara'] = item.findNext('span').findNext('span').get_text()

        for i in sp.findAll('span', attrs={'class': 'labelClass'}):
            entrada[i.get_text().replace(':', '')] = i.next.next.strip()

        # extinto_suspenso
        if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
            entrada['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

        if 'Distribuição' in entrada.keys():

            entrada['data_inicial'] = entrada['Distribuição'][:10]
        elif 'Recebido em' in entrada.keys():
            entrada['data_inicial'] = entrada['Recebido em'][:10]

        if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
            entrada['tramitacao_prioritaria'] = 'Tramitação prioritária'
        if 'Valor da ação' in entrada.keys():
            entrada['Valor da ação'] = re.sub(r'\s\s+', ' ', entrada['Valor da ação'])
            if re.match(r'(.*?)\s', entrada['Valor da ação']) is not None:
                entrada['moeda'] = re.match(r'(.*?)\s', entrada['Valor da ação']).group(1)
            if re.search(r'\s(.*)', entrada['Valor da ação']) is not None:
                rp = re.search(r'\s(.*)', entrada['Valor da ação']).group(1).strip()
                entrada['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

            del entrada['Valor da ação']

        # partes
        coleta = []
        controle = []

        for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top','style': 'padding-bottom: 5px'}):
            rp = {}
            if item.findNext('td').next.strip() not in controle:
                controle += [item.findNext('td').next.strip()]
                rp['nome'] = item.findNext('td').next.strip()
                rp['posicao'] = item.get_text().strip().replace(':', '')

                procuradores = []

                for subitem in item.findNextSibling().findAll('span'):
                    classe = subitem.get_text().replace(':', '').strip()
                    nome = subitem.next.next.strip()
                    procuradores += [[classe, nome]]

                classes = list(set([i[0] for i in procuradores]))
                subcoleta = {}

                for classe in classes:
                    nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                    subcoleta[classe] = nome_procurador

                if len(subcoleta) > 0:
                    rp['procuradores'] = subcoleta
                coleta += [rp]

        if len(coleta) > 0:
            entrada['partes'] = coleta

        # movimentacoes
        coleta = []
        lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
        r = list(range(len(lista) - 1, -1, -1))

        for n, item in enumerate(sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})[::-1]):

            mov = {'ordem': n,
                   'data': item.get_text().strip()}

            if len(item.findNext('td').findChildren()) > 0:
                mov['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
                mov['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
            else:

                mov['tag'] = item.findNext('td').findNext('td').next.strip()
                mov['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
            coleta += [mov]

        if len(coleta) > 0:
            entrada['movimentacoes'] = coleta

    entrada = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in entrada.items() if v}

    # formatacoes e salvar

    if 'data_inicial' in entrada.keys():

        entrada['data_inicial'] = datetime.datetime.strptime(entrada['data_inicial'], '%d/%m/%Y').date()


    dfp = pd.DataFrame([entrada])
    dfp = dfp.applymap(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x) 
    dfp.index = [_]
    df = df.append(dfp, sort=True)
    
    if _ % 250 == 0:
        df.to_csv(os.path.join(pasta, 'resultado_principais.csv'))
    
    print(_)
df.to_csv(os.path.join(pasta, 'resultado_principais.csv'))







13751
13752
13753
13754
13771
13772
13773
13774
13775
13776
13777
13778
13779
13780
13781
13782
13783
13784
13785
13786
13787
13813
13814
13815
13816
13817
13818
13868
13869
13870
13872
13873
13874
13884
13885
13920
13921
13922
13923
13924
13925
13926
13927
13928
13929
13930
13931
13932
13933
13934
13935
13936
13937
13938
13939
13940
13962
13963
13964
13965
13966
13967
13968
13969
13970
13971
13972
13973
13974
13975
13976
13977
13978
13979
13980
13981
13982
13983
13984
13985
13986
13987
14011
14012
14026
14029
14040
14041
14053
14055
14057
14058
14059
14060
14061
14062
14063
14064
14065
14066
14067
14068
14069
14070
14071
14072
14073
14094
14102
14103
14104
14105
14106
14107
14108
14109
14110
14111
14112
14113
14114
14115
14116
14117
14118
14119
14120
14121
14122
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14191
14192
14193
14197
14198
14199
14200
14201
14202
14203
14204
1420

16939
16940
16941
16942
16943
16944
16970
16971
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17040
17042
17051
17052
17053
17054
17055
17056
17057
17058
17059
17060
17061
17062
17063
17064
17065
17066
17067
17068
17069
17070
17071
17072
17084
17085
17086
17087
17088
17089
17090
17091
17092
17093
17094
17095
17096
17097
17098
17099
17100
17101
17102
17138
17182
17196
17197
17198
17199
17200
17201
17202
17203
17204
17205
17206
17207
17208
17209
17210
17211
17212
17213
17214
17215
17216
17248
17249
17250
17251
17252
17253
17254
17255
17256
17257
17258
17259
17260
17261
17262
17263
17264
17265
17266
17267
17268
17269
17270
17271
17272
17273
17274
17275
17276
17277
17278
17279
17280
17281
17282
17283
17284
17285
17286
17287
17288
17289
17290
17291
17292
17293
17294
17295
17296
17297
17298
17299
17300
17301
17302
17303
17304
17305
17306
17307
17308
17309
17310
17311
17312
17313
17314
17315
17316
17317
17318
17319
17320
17321
17322
17323
17324
17325
17326
1732

In [80]:
# df_a = pd.read_csv(os.path.join(pasta, 'resultado.csv'), index_col='Unnamed: 0')
# df_a = df_a[df_a['incidente_ou_execucao'].isnull()]
# df_a = df_a.append(df, sort=True)
# df_a.drop_duplicates(subset=['cnj'], inplace=True)
# del df_a['movimentacoes']

# del df_a['partes']
# w = pd.ExcelWriter(os.path.join(pasta, 'levantamento.xlsx'))
# df_a.to_excel(w)
# w.save()

In [50]:
#correcoes de cnj

from bs4 import BeautifulSoup as bs
import requests
import re
from unidecode import unidecode
import datetime
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

s = requests.session()


# df_cor = pd.read_excel(os.path.join(pasta, 'Levantamento Ezequiel - número de origem precatórios.xlsx'), index_col='Unnamed: 0')
df_cor = df_cor[df_cor['controle'].isnull()]

coleta = []

df_cor_f = pd.DataFrame()
    
for _, row in df_cor.iterrows():
    
    entrada = {'cnj':'0411408-71.1994.8.26.0053'}

    params = {
    'conversationId': '',
    'dadosConsulta.localPesquisa.cdLocal': '',
    'cbPesquisa': 'NUMPROC',
    'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
    'numeroDigitoAnoUnificado': '',
    'foroNumeroUnificado:': '',
    'dadosConsulta.valorConsultaNuUnificado': row['processo_principal'],
    'dadosConsulta.valorConsulta': '',
    'uuidCaptcha': '',
    }
    u = 'https://esaj.tjsp.jus.br/cpopg/search.do'

    r = s.get(u, params=params, verify=False, allow_redirects=True)

    sp = bs(r.content, 'html5lib')
    break
    if "mostrar: 'true'" in str(sp):
        entrada['segredo_de_justica'] = True

    else:
        # diversos

        for item in sp.findAll(attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
            if item.get('for') == 'tag.dados.rotulo.principal':
                entrada['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

            elif item.get_text() == 'Distribuição:':
                entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                rp = item.findNext('span').findNext()
                if rp.find('span') is not None:
                    entrada['vara'] = rp.find('span').get_text().strip()

            elif item.get_text() == 'Apensado ao:':
                entrada[item.get_text().strip().replace(':', '')] = item.findNext(
                    attrs={'class': 'processoPaiApenso'}).get_text().strip()

            elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
                entrada['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
            else:
                entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            if item.get_text() == 'Recebido em:':
                entrada['vara'] = item.findNext('span').findNext('span').get_text()

        for i in sp.findAll('span', attrs={'class': 'labelClass'}):
            entrada[i.get_text().replace(':', '')] = i.next.next.strip()

        # extinto_suspenso
        if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
            entrada['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

        if 'Distribuição' in entrada.keys():

            entrada['data_inicial'] = entrada['Distribuição'][:10]
        elif 'Recebido em' in entrada.keys():
            entrada['data_inicial'] = entrada['Recebido em'][:10]

        if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
            entrada['tramitacao_prioritaria'] = 'Tramitação prioritária'
        if 'Valor da ação' in entrada.keys():
            entrada['Valor da ação'] = re.sub(r'\s\s+', ' ', entrada['Valor da ação'])
            if re.match(r'(.*?)\s', entrada['Valor da ação']) is not None:
                entrada['moeda'] = re.match(r'(.*?)\s', entrada['Valor da ação']).group(1)
            if re.search(r'\s(.*)', entrada['Valor da ação']) is not None:
                rp = re.search(r'\s(.*)', entrada['Valor da ação']).group(1).strip()
                entrada['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

            del entrada['Valor da ação']

        # partes
        coleta = []
        controle = []

        for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top','style': 'padding-bottom: 5px'}):
            rp = {}
            if item.findNext('td').next.strip() not in controle:
                controle += [item.findNext('td').next.strip()]
                rp['nome'] = item.findNext('td').next.strip()
                rp['posicao'] = item.get_text().strip().replace(':', '')

                procuradores = []

                for subitem in item.findNextSibling().findAll('span'):
                    classe = subitem.get_text().replace(':', '').strip()
                    nome = subitem.next.next.strip()
                    procuradores += [[classe, nome]]

                classes = list(set([i[0] for i in procuradores]))
                subcoleta = {}

                for classe in classes:
                    nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                    subcoleta[classe] = nome_procurador

                if len(subcoleta) > 0:
                    rp['procuradores'] = subcoleta
                coleta += [rp]

        if len(coleta) > 0:
            entrada['partes'] = coleta

        # movimentacoes
        coleta = []
        lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
        r = list(range(len(lista) - 1, -1, -1))

        for n, item in enumerate(sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})[::-1]):

            mov = {'ordem': n,
                   'data': item.get_text().strip()}

            if len(item.findNext('td').findChildren()) > 0:
                mov['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
                mov['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
            else:

                mov['tag'] = item.findNext('td').findNext('td').next.strip()
                mov['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
            coleta += [mov]

        if len(coleta) > 0:
            entrada['movimentacoes'] = coleta

    entrada = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in entrada.items() if v}

    # formatacoes e salvar

    if 'data_inicial' in entrada.keys():

        entrada['data_inicial'] = datetime.datetime.strptime(entrada['data_inicial'], '%d/%m/%Y').date()

    dfp = pd.DataFrame([entrada])
    dfp = dfp.applymap(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x) 
    dfp.index = [_]
    df_cor_f = df_cor_f.append(dfp, sort=True)
    print(_)




In [73]:
p = {'conversationId': '',
'dadosConsulta.localPesquisa.cdLocal': '53',
'cbPesquisa': 'NUMPROC',
'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
'numeroDigitoAnoUnificado': '0405535-22.1996',
'foroNumeroUnificado': '0053',
'dadosConsulta.valorConsultaNuUnificado': '04055352219968260053',
'dadosConsulta.valorConsulta': '',
'uuidCaptcha': '',}

r = s.get(u, params=params, verify=False, allow_redirects=True)

sp = bs(r.content, 'html5lib')
sp

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>

	








	<script language="javascript" src="https://esaj.tjsp.jus.br/sajcas/verificarLogin.js?script=1570586384800" type="text/JavaScript"></script>

	<script language="javascript">
		if(window.sajcas && window.sajcas.usuarioLogadoNoCasServer) {
			var urlRetornoSistema = '';
			if(urlRetornoSistema === '') {
				urlRetornoSistema = window.location.href;
			}

			if(urlRetornoSistema.lastIndexOf('?') != -1) {
				urlRetornoSistema += '&';
			} else {
				urlRetornoSistema += '?';
			}
			urlRetornoSistema+= 'gateway=true';
			window.location.href = urlRetornoSistema;
		}
	</script>


	<title>Portal de Serviços e-SAJ</title>
	































<!-- Includes do Select2 -->

<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,